<a href="https://colab.research.google.com/github/jeonghaerin/breed_classification/blob/main/our_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 두 가지 카테고리 분류
# zzllrr imagergeek 다운로드
# 수집하고 싶은 이미지 검색
# 우측 상단 아이콘 클릭 -> Go 버튼 누른후 화면 전환
# 필요 없는 이미지, 아이콘 파일 클릭해서 제거
# rename 버튼 클릭후 prefix버튼 클릭 이름 명명 규칙 수정
# 수정한 이미지 이름 preview 버튼 클릭후 반영 확인
# save 누른 후 수집 대기 -> 수집 완료 후
# 해당 카테고리 폴더 생성후 모두 저장
# 상위 폴더 생성후 카테고리 폴더를 모두 넣기
# 상위 폴더 압축 -> our_dataset.zip

In [ ]:
# drive에 압축파일 업로드 하고
# 아래 코드 실행
!unzip /content/our_dataset.zip

Archive:  /content/our_dataset.zip
replace our_dataset/japanese_spitz/japanese_spitz-001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 파일을 접근 -> 메모리에 파일을 올려야 -> 모든 파일의 경로가 필요
from glob import glob
mother_path = '/content/our_dataset'
# dataset 폴더가 없는 상태에서 풀렸다면
# mother_path = '/content/
glob(mother_path + '/*/*')

In [ ]:
import os
a = '/content/'
b = 'our_dataset'
os.path.join(a,b) # = a+'/'+b 리눅스, 맥, window os마다 다름

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
folder_list = ['japanese_spitz', 'pomeranian']
train = []
label = []
for idx , folder_name in enumerate(folder_list):
    folder = os.path.join(mother_path, folder_name)
    all_files_path = glob(folder+'/*')

    for k,file_path in enumerate(all_files_path):
        # 파일 불러오기
        img = Image.open(file_path)
        # 이미지 파일 전처리
        img = img.resize((128,128))
        img = img.convert('RGB')
        np_img = np.asarray(img)
        train.append(np_img)
        label.append(idx)
        if k % 20 == 0:
            print(f'{k} / {len(all_files_path)}')
np_train = np.array(train)
np_label = np.array(label)

In [ ]:
print(np.bincount(np_label))
print(np_train.shape)
#np.save()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np_train, np_label, test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#xy = (x_train, y_train)
#np.save('data.npy',xy)
#np.load('data.npy')

In [ ]:
x_train = x_train / 255.
x_test = x_test / 255.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_layer = layers.Input(shape=(128,128,3), name='start layer')
tower_1 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_1' )(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_1')(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2')(tower_2)
tower_3 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_1')(input_layer)
tower_3 = layers.Conv2D(64 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_2')(tower_3)
tower_4 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_4_1')(input_layer)
tower_4 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_4_2')(tower_4)
#
concat_layer = layers.concatenate([tower_1,tower_2,tower_3,tower_4],axis=3)
concat_layer = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer)
##
tower_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_2_1_1' )(concat_layer)
tower_2_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_2_1')(concat_layer)
tower_2_2_2 = layers.Conv2D(120 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2_2')(tower_2_2_1)
tower_2_3_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_1')(concat_layer)
tower_2_3_2 = layers.Conv2D(120 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_2')(tower_2_3_1)
tower_2_4_1 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_2_4_1')(concat_layer)
tower_2_4_2 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_4_2')(tower_2_4_1)
concat_layer_2 = layers.concatenate([tower_2_1,tower_2_2_2,tower_2_3_2,tower_2_4_2],
                                  axis=3)
concat_layer_2 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_2)
#
last_avg_pool = layers.AveragePooling2D(pool_size=(31,31) , strides=(1,1), padding='valid')(concat_layer_2)
flat_layer = layers.Flatten()(last_avg_pool)
output_layer = layers.Dense(1 ,activation='sigmoid')(flat_layer)

# 변경 포인트 input, last_avg_pool,output layer
model = models.Model(input_layer,output_layer)
model.summary()

In [ ]:
!mkdir tensorboard_log

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
tb = tf.keras.callbacks.TensorBoard(log_dir='/content/tensorboard_log',
                                    histogram_freq=1)
#
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          epochs=10,callbacks=[tb])

In [ ]:
from google.colab import files
f = files.upload()

In [ ]:
test_file_name = list(f.keys())[0]

In [ ]:
img = Image.open(file_path)
img = img.resize((128,128))
img = img.convert('RGB')
data = np.asarray(img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape

In [ ]:
result = model.predict(test_image_tensor)
if result[0][0] > 0.5:
  print(f'{folder_list[1]} 일 확률이 {(1 - result[0][0]) * 100} % 입니다.')
else:
  print(f'{folder_list[0]} 일 확률이 {result[0][0] * 100} % 입니다.')


## Normalize를 하는 이유

통계 및 통계 응용에서 정규화는 다양한 의미를 가질 수 있습니다. 가장 단순한 경우에, 등급의 정규화는 종종 평균화하기 전에 다른 척도에서 측정된 값을 개념적으로 일반적인 척도로 조정하는 것을 의미합니다.